# SageMaker JumpStart を用いた物体検出転移学習
Builders Online Series "Amazon SageMaker JumpStart を用いて機械学習 PoC を IT エンジニアの手で実行する ～ タケノコ好きに向けたキノコ検出問題を添えて ～"  
で登壇した内容を再現するコードです。  
前提として、SageMaker Studio で、このノートブックを開き、Kernelは`Data Science`、インスタンスは`ml.t3.medium`を選択します。  

## ディレクトリ構成
* manifest
    * output.manifest  
    `train_raw_images/*.jpg`をSageMaker GroundTruth でラベリングした結果。自身でSageMaker GroundTruth でラベリングする際や、別途用意した画像を使う場合はこのファイルは不要。  
    `train_raw_images/*.jpg` をそのまま使い、ラベリングをしたくない場合は使用する
* test_crop_images
  `test_raw_images/takenoko.jpg`から 512px x 512px の画像をスライドさせながら切り出した画像を格納するフォルダ。  
  `8-2. ベルトコンベアを模した推論`で使用する  
  初期状態では空ディレクトリ
* test_detect_images
  `test_crop_images/*.png`にある画像を推論した結果を保存するディレクトリ
  `8-2. ベルトコンベアを模した推論`で使用する  
  初期状態では空ディレクトリ
* test_raw_images
  テスト用の画像を配置  
  `latitice.png` は格子状にお菓子を配置した画像で、`8-1. 格子状に配置した画像を推論する`で使用する  
  `takenoko.jpg` はお菓子を直線に配置した画像で、`8-2. ベルトコンベアを模した推論`で使用する  
* train_random_crop_images
  学習用にcropした画像を配置する  
  `5. ラベルと学習データの整形 ` で使用する
  初期状態では空ディレクトリ
* train_raw_images
  学習用の生画像  

## 手順
* [1. 使用するモジュールのインストールと読み込み](#1.-使用するモジュールのインストールと読み込み)
* [2. 画像収集](#2.-画像収集)
* [3. ラベリングジョブを作成](#3.-ラベリングジョブを作成)
* [4. ラベリング](#4.-ラベリング)
* [5. ラベリング結果をダウンロード](#5.-ラベリング結果をダウンロード)
* [6. ラベルと学習データの整形](#6.-ラベルと学習データの整形)
* [7. 学習する](#7.-学習する)
* [8. デプロイする](#8.-デプロイする)
* [9. 推論する](#9.-推論する)
  * [9-1. 格子状に配置した画像を推論する](#9-1.-格子状に配置した画像を推論する)
  * [9-2. ベルトコンベアを模した推論](#9-2.-ベルトコンベアを模した推論)

## 1. 使用するモジュールのインストールと読み込み
* この Notebook で利用するモジュールのインストールと読み込み
  * SageMaker Studio で Data Science カーネルを前提としている
  * OpenCV が入っていないためインストールする
* 最後に再現できるように seed を固定しておく

In [2]:
!apt-get update && apt-get upgrade -y
!apt-get install libgl1-mesa-dev -y
!pip install opencv-python

import sagemaker, json, numpy as np, os, boto3, uuid
from PIL import Image, ImageDraw, ImageOps, ImageColor, ImageFont
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from glob import glob
import cv2
np.random.seed(seed=1234)

Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [309 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7906 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [15.2 kB]
Fetched 8469 kB in 2s (5059 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages will be upgraded:
  apt base-files ca-certificates debconf debian-archive-keyring file
  krb5-locales libapt-pkg5.0 libbsd0 libcurl3-gnutls libgcrypt20 libglib2.0-0
  libglib2.0-data libgnutls30 libhogweed4 libk5crypto3 libldap-2.4-2
  libldap-common liblz4-1 libmagic-mgc libmagic1 libnettle6 libp11-kit0
  libsqlite3-0 libssl1.

## 2. 画像収集
* 今回は`./train_raw_images`に格納済
* このリポジトリに含まれる画像を使う場合は下記セルでファイルの有無を確認する
* 自身のデータを利用するには`./train_raw_images/*.jpg`にある画像を差し替えておく

In [3]:
# ファイル有無確認
TRAIN_RAWIMAGE_DIR = './train_raw_images/'
print(*sorted(glob(TRAIN_RAWIMAGE_DIR+'*.jpg')))

./train_raw_images/01.jpg ./train_raw_images/02.jpg ./train_raw_images/03.jpg ./train_raw_images/04.jpg ./train_raw_images/05.jpg ./train_raw_images/06.jpg ./train_raw_images/07.jpg ./train_raw_images/08.jpg ./train_raw_images/09.jpg ./train_raw_images/10.jpg ./train_raw_images/11.jpg ./train_raw_images/12.jpg ./train_raw_images/13.jpg ./train_raw_images/14.jpg ./train_raw_images/15.jpg ./train_raw_images/16.jpg ./train_raw_images/17.jpg ./train_raw_images/18.jpg ./train_raw_images/19.jpg ./train_raw_images/20.jpg ./train_raw_images/21.jpg ./train_raw_images/22.jpg ./train_raw_images/23.jpg


## 3. SageMaker GroundTruth でラベリング
### 3-1. ラベリング対象を S3 にアップロード
* SageMaker GroundTruth でラベリングをする際は、予め S3 にアップロードする必要がある
* 以下はアップロードする(とアップロード先に事前にファイルがあった場合の削除する)コマンド
* `print()`の出力結果は SageMaker GroundTruth でラベリングジョブを作成するのに使用する

In [4]:
BASE_PREFIX = 'takenoko_kinoko_gt'
GT_JOB_NAME = f'{BASE_PREFIX}-{uuid.uuid4()}'.replace('_','-')
bucket = sagemaker.session.Session().default_bucket()
!aws s3 rm s3://{bucket}/{BASE_PREFIX} --recursive
rawimage_s3_uri = sagemaker.session.Session().upload_data(TRAIN_RAWIMAGE_DIR,key_prefix=BASE_PREFIX)

In [5]:
print(f'GroundTruth job name : {GT_JOB_NAME}')
print(f'GroundTruth Target : {rawimage_s3_uri}/')

GroundTruth job name : takenoko-kinoko-gt-73550b23-689a-4fc9-8c22-6b5d8e8eef4b
GroundTruth Target : s3://sagemaker-ap-northeast-1-253385185774/takenoko_kinoko_gt/


## 3. ラベリングジョブを作成

### 3-1. ラベリング用のプライベートチームを作成
1. [ラベリングワークフォースの画面](https://ap-northeast-1.console.aws.amazon.com/sagemaker/groundtruth?region=ap-northeast-1#/labeling-workforces)にアクセスし、`プライベート` タブをクリックし、`プライベートチームを作成` をクリック
  上記リンクは`ap-northeast-1`に飛ぶリンクなので、違うリージョンを使っている場合は適宜修正
2. `AWS Cognito` でプライベートチームを作成するにチェックを入れ、チーム名に任意の名前(ex:kinoko-detect-team)を入力し、`E メールによる新しいワーカーの招待` にチェックを入れ、 `E メールアドレス`にラベリングさせたい人のメールアドレスをカンマ区切りで入力、`組織名`に任意の名前(ex:茸探隊)を入力、`連絡先 E メール` にチームを管理する人のメールアドレスを入力し、`プライベートチームを作成`をクリック
3. プライベートワークフォースの概要の画面に `ラベリングポータルのサインイン URL` が表示されるので、そちらをクリックする。
4. しばらくすると別途ワーカー宛にメールが送信されるので、そこに書いてあるパスワードをコピーする
5. 3 で出現するログイン画面にワーカーのメールアドレスとメールで送られてきたパスワードを入力し、 `Sign in` をクリック
6. パスワード変更を求められるので新しいパスワードを入力し、`Send` をクリック
7. ラベリングジョブのポータルに遷移する。まだラベリングジョブが無いのでラベリングジョブを作成する

### 3-2. ラベリングジョブの作成

1. [SageMaker GroundTruth のラベリングジョブ作成画面](https://ap-northeast-1.console.aws.amazon.com/sagemaker/groundtruth?region=ap-northeast-1#/labeling-jobs)にアクセスし、`ラベリングジョブの作成` をクリック
2. `ジョブ名`に上のセルの `GroundTruth job name :` 以降の文字列を記入
3. `入力データセットの S3 の場所`に上のセルの`GroundTruth Target :`以降の文字列を記入
4. `出力データセットの S3 の場所` はデフォルトのまま`入力データセットと同じ場所` を選択
5. `データタイプ`は`画像`を選択
6. `IAM ロール`は今動かしている SageMaker Studio と同じロールを選択
7. `完全なデータセットアップ`をクリック
8. `タスクの選択`で、`境界ボックス`を選択
9. `次へ`をクリック
10. `ワーカータイプ`で`プライベートチーム`を選択
11. `プライベートチーム`のプルダウンで作成済のワーカーを選択(要事前に作成)
12. 他はデフォルトのまま、`境界ボックスラベリングツール`の部分で、`take`と`kino`というラベルを作成
    初期の空白ラベルは１つなので、`take`を入力後、`新しいラベルを追加`をクリックして、`kino` を入力
13. `作成`をクリック

## 4. ラベリング
プライベートチームにログインして、タケノコ状のお菓子と、キノコ状のお菓子をそれぞれ矩形で括る

## 5. ラベリング結果をダウンロード
* ラベリング結果である`output.manifest`をダウンロードする。
    * このリポジトリにはあらかじめ`output.manifest`を`./manifest/output.manifest`に用意しているので、ラベリングを割愛したい場合はそちらを使う(下のセルはコメントアウトのまま)
    * ラベリングをした場合は下のセルのコメントアウトを外す

In [ ]:
# sagemaker.session.Session().download_data('./manifest/',key_prefix=f'{BASE_PREFIX}/{GT_JOB_NAME}/manifests/output/output.manifest',bucket=bucket)

## 6. ラベルと学習データの整形
* SSD MobileNet 1.0 で転移学習するために`./manifest/output.manifest`を加工して、`manifest.json`を作成する
* データ形式は、以下を遵守する必要がある
```
The annotations.json file should should have information for bounding_boxes and their class labels. It should have a dictionary with keys "images" and "annotations". Value for the "images" key should be a list of entries, one for each image of the form {"file_name": image_name, "height": height, "width": width, "id": image_id}. Value of the 'annotations' key should be a list of entries, one for each bounding box of the form {"image_id": image_id, "bbox": [xmin, ymin, xmax, ymax], "category_id": bbox_label}.
```
* 学習データと`manifest.json`はS3に最終的に配置する必要があるが、S3でのデータの配置は以下を遵守する必要がある
```
input_directory
    |--images
        |--abc.png
        |--def.png
    |--annotations.json     
```
* 学習データは SSD MobileNet 1.0 は 512 x 512 の画像サイズにする必要がある
  * 学習用の生データは 1147 x 1108 のため、お菓子が少なくとも 2 個以上写っている場所をランダムに 512 x 512 で切り取る
  * 切り取る枚数は書く画像から 5 枚
  * ラベリングでくくった矩形の面積の 1/4 以上写っていればお菓子が写った、とカウントする
* 512 x 512 に切り取った画像は、ラベリングの情報と当然ずれるので、切り取った座標位置を用いて補正する


これらの処理を以下で行い、512 x 512 の画像を`./train_random_crop_images`に、ラベリングデータを`./manifest.json`に出力し、それらを`s3://sagemaker-{region}-{account}/train_randcrop`以下に保存する

In [ ]:
def fix_bbox(l,t,r,b,w,h):
    judge = True
    
    fix_left = 0 if l < 0 else l
    fix_top = 0 if t < 0 else t
    fix_right = w if r > w else r
    fix_bottom = h if b > h else b
    
    # 領域外なら
    if l > w or t > h or r < 0 or b <0:
        judge=False
    # 基の面積の1/4以下ならアノテーション無しとする
    elif (r-l)*(b-t)/4 > (fix_right-fix_left)*(fix_bottom-fix_top):
        judge=False
    
    return judge,(fix_left,fix_top,fix_right,fix_bottom)
    

In [ ]:
with open('manifest/output.manifest','r') as f:
    manifest_line_list = f.readlines()
annotation_dict = {
    'images':[],
    'annotations':[]
}
IMAGE_SIZE_TUPLE=(512,512)
IMAGE_ID = 0

OUTPUT_DIR = './train_random_crop_images/'
!rm -rf {OUTPUT_DIR}*.png

for i,manifest_line in enumerate(manifest_line_list):
    manifest_dict = json.loads(manifest_line)
    filename = manifest_dict['source-ref'].split('/')[-1]
    image_size_tuple=(manifest_dict['kinoko-takenoko-aug']['image_size'][0]['width'],manifest_dict['kinoko-takenoko-aug']['image_size'][0]['height'])
    raw_img = Image.open(os.path.join(TRAIN_RAWIMAGE_DIR,filename))
    for i in range(20):
        loop = True
        while loop:
            rand_x = np.random.randint(0,image_size_tuple[0]-IMAGE_SIZE_TUPLE[0])
            rand_y = np.random.randint(0,image_size_tuple[1]-IMAGE_SIZE_TUPLE[1])
            crop_img = raw_img.crop((
                rand_x,
                rand_y,
                rand_x + IMAGE_SIZE_TUPLE[0],
                rand_y + IMAGE_SIZE_TUPLE[1]
            ))
            annotation_list = []
            for annotation in manifest_dict['kinoko-takenoko-aug']['annotations']:
                left = annotation['left'] - rand_x
                top = annotation['top'] - rand_y
                right = annotation['left'] + annotation['width'] - rand_x
                bottom = annotation['top'] + annotation['height'] - rand_y
                judge,(left,top,right,bottom) = fix_bbox(left,top,right,bottom,IMAGE_SIZE_TUPLE[0],IMAGE_SIZE_TUPLE[1])
                if judge:
                    annotation_list.append(
                        {
                            'bbox':[left,top,right,bottom],
                            'category_id':annotation['class_id']
                        }
                    )
            if len(annotation_list) > 1:
                loop = False
        
        save_file_name = f'{str(IMAGE_ID).zfill(5)}_{str(i).zfill(5)}_{filename}'.replace('jpg','png')
        crop_img.save(os.path.join(OUTPUT_DIR,save_file_name))
        annotation_dict['images'].append(
            {
                'file_name' : save_file_name,
                'height' : IMAGE_SIZE_TUPLE[1],
                'width' : IMAGE_SIZE_TUPLE[0],
                'id' : IMAGE_ID
            }
        )
        for annotation in annotation_list:                  
            annotation_dict['annotations'].append(
                {
                    'image_id': IMAGE_ID,
                    'bbox':annotation['bbox'],
                    'category_id':annotation['category_id']
                }
            )
        IMAGE_ID += 1

with open('annotations.json','wt') as f:
    f.write(json.dumps(annotation_dict))        

prefix = OUTPUT_DIR[2:-1]
!aws s3 rm s3://{bucket}/{prefix} --recursive
        
image_s3_uri = sagemaker.session.Session().upload_data(OUTPUT_DIR,key_prefix=f'{prefix}/images')
annotatione_s3_uri = sagemaker.session.Session().upload_data('./annotations.json',key_prefix=prefix)
paste_str = image_s3_uri.replace('/images','')
print(f"paste string to S3 bucket address：{paste_str}")

## 7. 学習する
SageMaker JumpStart で転移学習を行う

1. SageMaker Studio の左上にある `+` アイコンなどから  Launcher タブを出し、`Get started` の`Explore one-click solutions, models, and tutorials SageMaker JumpStart`の中にある`Go to SageMaker JumpStart` をクリックする  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/launch_get_started.png)  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/go_to_sagemaker_jumpstart.png)  
2. `Search` と書かれた検索窓に `SSD MobileNet` と打ち込むと青い丸に`m`と書かれたアイコンで SSD MobileNet 1.0 と表示されるので、それをクリックする  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/ssd_mobilenet.png)  
3. やや下にあるFine-tune Model のData Source のラジオボタンを `Enter S3 bucket location` を選択し、`S3 bucket address` に上のセルの `paste string to S3 bucket address: `より後ろの値を貼り付ける
4. `Deployment Configuration` をクリックし、`SageMaker Training Instance` で `ML.G4dn.xlarge`を選択する。`Model Name` は任意の名前を入力する（デフォルトのままでも可だが、わからなくなりやすいので、検出したいモノなどを入れるとよい。例：`kinoko-detection-model` など）
5. `Hyper-parameters`をクリックし、下記を入力する。（デフォルトのままだと学習が短く急なため変更）
    * learning rate : 0.0001
    * batch-size : 4
    * epochs : 40  
6. `Train` をクリックして転移学習をスタートさせる。

![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/train_specification.png)  

## 8. デプロイする
完了するとデプロイできるようになるのでデプロイする。
1. `Deployment Configuration`をクリックする
2. `SageMaker Hosting Instance`のプルダウンで`ML.M5.Large`を選択する
3. Endpoint Name に任意の名前を入力する(ex: `kinoko-detection-endpoint`)
4. `Deploy` をクリックする  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/deploy_specification.png)  
5. In Serviceに変わることを確認する(デプロイ完了確認)  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/deploy_completed.png)  

## 9. 推論する
### 9-1. 格子状に配置した画像を推論する
* 出来上がったモデルで推論を行う
* request の Body に `open` でメモリに展開したバイナリ情報を入れて推論する
* `ENDPOINT_NAME` 変数に上の画像の Endpoint Name の文字列を入力して、エンドポイントを指定する
* `test_raw_images/lattice.png`を利用して、推論する。推論結果も併せて画像に矩形を描画して可視化する

In [ ]:
smr_client = boto3.client('sagemaker-runtime')
ENDPOINT_NAME='jumpstart-ftc-kinoko-detection-endpoint'
TEST_IMAGE_FILE = 'test_raw_images/lattice.png'

In [ ]:
# 推論対象の画像を開いて変数に格納
with open(TEST_IMAGE_FILE, 'rb') as f:
    img_bin = f.read()

# 推論を実行
response = smr_client.invoke_endpoint(EndpointName=ENDPOINT_NAME, ContentType='application/x-image', Body=img_bin)

# 推論結果を読み込む
model_predictions = json.loads(response['Body'].read())

# 結果を可視化
colors = list(ImageColor.colormap.values())
image_np = np.array(Image.open(TEST_IMAGE_FILE))
fig = plt.figure(figsize=(20,20))
ax = plt.axes()
ax.imshow(image_np)
bboxes, classes, confidences = model_predictions
for idx in range(len(bboxes)):
    if confidences[idx]>0.6:
        left, bot, right, top = bboxes[idx]
        x, w = [val * image_np.shape[1] for val in [left, right - left]]
        y, h = [val * image_np.shape[0] for val in [bot, top - bot]]
        color = colors[hash(classes[idx]) % len(colors)]
        class_name = 'take' if classes[idx]=='0' else 'kino'
        color = 'blue' if class_name == 'take' else 'red'
        rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(class_name, confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))

### 9-2. ベルトコンベアを模した推論
* `./test_raw_images/takenoko.jpg` にタケノコが横一列に並んでいる（１つだけキノコが混在）
* 512x512の画像をスライドしながら切り出すことでベルトコンベアでお菓子が流れているような動画として扱う
* 各画像に対して推論をかけ、最後に１つの動画として出力する

In [ ]:
x ,y = 0,512
CROP_SIZE=(512,512)
img = Image.open('./test_raw_images/takenoko.jpg')
CROP_DIR = './test_crop_images/'
!rm -rf {CROP_DIR}/*.png
for i in range(img.size[0]-CROP_SIZE[0]):
    crop_img = img.crop((i,y,i+CROP_SIZE[0],y+CROP_SIZE[1]))
    file_name = f'{CROP_DIR}{str(i).zfill(5)}.png'
    crop_img.save(file_name)

全データ推論して、矩形を描いた画像を生成する

In [ ]:
TEXTSIZE=14
LINEWIDTH=4
DETECT_DIR='./test_detect_images/'
!rm -rf {DETECT_DIR}/*.png

smr_client=boto3.client('sagemaker-runtime')
for img_file_path in sorted(glob(f'{CROP_DIR}*.png')):

    with open(img_file_path,'rb') as f:
        img_bin = f.read()
    response = smr_client.invoke_endpoint(EndpointName=ENDPOINT_NAME, ContentType='application/x-image', Body=img_bin)
    pred=json.loads(response['Body'].read())
    bboxes, classes, confidences = pred
    img = Image.open(img_file_path)
    bboxes, classes, confidences = pred
    draw = ImageDraw.Draw(img)
    for i in range(len(bboxes)):
        if confidences[i]>0.7:
            left, top, right, bottom = bboxes[i]
            left = img.size[0] * left
            top = img.size[1] * top
            right = img.size[0] * right
            bottom = img.size[1] * bottom
            text = 'take' if classes[i]=='0' else 'kino'
            color = 'blue' if text == 'take' else 'red'
            TEXTSIZE=14 if classes[i]=='0' else 18
            LINEWIDTH=4 if classes[i]=='0' else 6
            draw.rectangle([(left,top),(right,bottom)], outline=color, width=LINEWIDTH)
            text += f' {str(round(confidences[i],3))}'
            txpos = (left, top-TEXTSIZE-LINEWIDTH//2)
            font = ImageFont.truetype("/usr/share/fonts/truetype/noto/NotoMono-Regular.ttf", size=TEXTSIZE)
            txw, txh = draw.textsize(text, font=font)
            draw.rectangle([txpos, (left+txw, top)], outline=color, fill=color, width=LINEWIDTH)
            draw.text(txpos, text, fill='white',font=font)
    img.save(img_file_path.replace(CROP_DIR,DETECT_DIR))

画像を動画に書き込む

In [ ]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter('./video.mp4',fourcc, 120.0, CROP_SIZE)
for img_file_path in sorted(glob(f'{DETECT_DIR}*.png')):
    img = cv2.imread(img_file_path)
    video.write(img)
video.release()